# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key



import jsonimport json### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
city_weather_df=pd.read_csv("../WeatherPy/city_weather_df.csv")
city_weather_df

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloud coverage,Wind Speed,City datetime
0,Codrington,AU,-38.2667,141.9667,53.28,89,92,5.46,1617139429
1,Kinston,US,35.2627,-77.5816,75.20,43,40,5.75,1617139429
2,Shemgang,BT,27.2169,90.6579,48.07,70,49,4.47,1617139429
3,Mataura,NZ,-46.1927,168.8643,55.99,98,96,3.00,1617139430
4,Hithadhoo,MV,-0.6000,73.0833,83.41,75,76,18.05,1617139430
...,...,...,...,...,...,...,...,...,...
496,Östersund,SE,63.1792,14.6357,35.60,69,20,16.11,1617139559
497,Paso de Carrasco,UY,-34.8603,-56.0522,70.00,60,40,10.36,1617139560
498,Minbu,MM,20.1833,94.8833,77.02,55,6,4.47,1617139560
499,Châtellerault,FR,46.8000,0.5333,60.80,44,0,3.44,1617139560


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
#create list for lat and long
humidity = city_weather_df["Humidity"].astype(float)
max_hum=humidity.max()
locations=city_weather_df[["Latitude", "Longitude"]]

In [17]:
#Map using gmaps
fig = gmaps.figure()

#add heat layer to map
heat_layer=gmaps.heatmap_layer(locations,weights=humidity, dissipating=False, 
                                   max_intensity=10, point_radius=1)
# Add layer
fig.add_layer(heat_layer)

#print
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
#Perfect weather conditions, create final df and drop null values 

weather_pref_df = city_weather_df.loc[(city_weather_df["Max Temperature"]>=68) & (city_weather_df["Max Temperature"]<=90)]
wind_pref_df = weather_pref_df.loc[(weather_pref_df["Wind Speed"]<=5)]
cloud_pref_df = wind_pref_df.loc[(wind_pref_df["Cloud coverage"]<5)]
ideal_weather_df = cloud_pref_df.dropna(how = "any")
ideal_weather_df

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloud coverage,Wind Speed,City datetime
40,Gerāsh,IR,27.6652,54.1371,69.80,23,0,4.68,1617139439
117,Makkah al Mukarramah,SA,21.4267,39.8261,76.06,59,0,1.92,1617139457
190,Saint-Philippe,RE,-21.3585,55.7679,78.80,73,0,3.44,1617139476
216,Santa Maria,BR,-29.6842,-53.8069,77.00,61,0,4.61,1617139452
229,Big Spring,US,32.2504,-101.4787,82.00,13,1,4.61,1617139350
330,Saint-Pierre,RE,-21.3393,55.4781,78.80,73,0,3.44,1617139516
339,Kununurra,AU,-15.7667,128.7333,69.80,88,4,4.61,1617139518
394,Mercedes,AR,-34.6515,-59.4307,73.00,52,0,3.00,1617139532
464,Morón,AR,-34.6534,-58.6198,73.00,56,0,1.99,1617139551
485,Saint-Joseph,RE,-21.3667,55.6167,78.80,73,0,3.44,1617139556


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
#Create a dataframe called hotel_df to store hotel names along with city, country and coordinates

#store in df 
hotel_df = ideal_weather_df.reset_index(drop=True)

#add hotel name column to df
hotel_df["Hotel Name"]= " "

# geocoordinates
target_radius = 5000
target_type = "hotel"
target_search = "hotel"

params={
    "radius": target_radius, 
    "types" : target_type, 
    "hotel" : target_search, 
    "key" : g_key  
}

for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    long = row["Longitude"]
    
    params["location"]=f"{lat}, {long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)
    
    hotel_df = response.json()
    
    print(json.dumps(hotel_df, indent=4, sort_keys=True))


{
    "html_attributions": [],
    "next_page_token": "ATtYBwLH-g41q_p5bcVZIjLHjO4iOpBQhMFgnPt-nW0JGEExoQQvuyyh3nnn8MNV1HW0c8bWaFxS_aeFRIB36H-qs2pDoR5tQmY_FhHkogGqWoaIQGKAimDf6d5wcOEkkUZWK3yHHQxw6ZZAcJ5Tlw3sVTe1ojMIbwIzWQQRXF2ngjt-XnKTZt2CRzPLjMSlKm-s1aX3avbPl-_yJ1WifNHccc2Q-xWDQaSRw3IbmAHFkhRMwnsj2ln37aIj0ujiEXgaZ70IcJhV7wD6SwMm7LK0j8-V74QU9CJkTz_K0QXHMe78e2dp_in37YrxtrgmgUVpm9SK4DlzF2rCuB806S5yoRQmA2bORgjOzS1OH__PSViBb9s7yqzA8pRRSRYeXYH2ofKGXbk1RdhvFftLPrELDg8QeOxAS-2vMPwTw14GPtVtoEDh2TAujg",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 27.6903817,
                    "lng": 54.1501922
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.7358073,
                        "lng": 54.2438793
                    },
                    "southwest": {
                        "lat": 27.6141892,
                        "lng": 54.050535
                    }
       

In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

AttributeError: 'dict' object has no attribute 'iterrows'

In [22]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig


# Display figure

Figure(layout=FigureLayout(height='420px'))